# Production Scheduling
Pada sesi ini kita akan menggunakan library pulp untuk melakukan optimasi penjadwalan produksi. Tujuan optimasi ini yaitu menentukan jumlah produk yang akan diproduksi dan disimpan pada setiap periode supaya dapat meminimalkan biaya. Permasalahan ini saya dapatkan dari kursus udemy **Supply Chain Design and Planning with Excel & Python - Haytham Omar**

### 1. Mengimpor Library
Library yang dibutuhkan yakni:
- pulp: melakukan optimasi linear programming
- numpy: membantu perhitungan array/matriks (opsional)
- pandas: membuka dan menyimpan data tabular

In [4]:
import pandas as pd
import numpy as np
from pulp import *

### 2. Mengimpor Data 

Data yang digunakan yakni **Production_scheduling.xlsx** yang terdiri dari beberapa kolom sebagai berikut:
- period: menunjukan waktu produksi (bulan)
- demand: jumlah produk yang harus dipenuhi setiap periode
- fixed cost: biaya tetap yang dibutuhkan apabila kegiatan produksi **dilakukan** pada periode tersebut (dalam satuan dolar)
- var: biaya variabel per unit produk (dalam satuan dolar)
- storage cost: biaya penyimpanan per unit produk (dalam satuan dolar)
- capacity: kapasitas jumlah produksi setiap periode

In [5]:
param = pd.read_excel("Production_scheduling.xlsx")
param.rename(columns={'Unnamed: 0':'period'}, inplace=True)
param['t'] = range(1,13)
param.set_index('t', inplace=True)
param

,period,demand,fixed cost,var,storage cost,Capacity
t,,,,,,
1,period 1,3000,2000,40,1,4000
2,period 2,4000,2000,40,1,4000
3,period 3,2500,2000,40,1,4000
4,period 4,4000,2000,40,1,4000
5,period 5,6000,2000,40,1,4000
6,period 6,100,2000,40,1,4000
7,period 7,250,2000,40,1,4000
8,period 8,5000,2000,40,1,4000
9,period 9,6000,2000,40,1,4000


### 3. Menentukan Variabel
Variabel yang akan kita butuhkan antara lain:
- inventory: variabel yang menunjukkan jumlah unit yang akan **disimpan** pada periode ke i
- production: variabel yang menunjukkan jumlah unit yang akan **diproduksi** pada periode ke i
- binary: variabel yang menyatakan apakah akan dilakukan produksi atau tidak pada periode ke i. Apabila dilakukan produksi maka akan dikenakan biaya tetap.

Permasalahan ini juga memiliki asumsi kapasitas, jenis biaya yang konstan serta **di awal periode 1 sudah tersedia 200 unit produk**

In [6]:
#define variable
inventory = LpVariable.dicts('inv', list(range(0,13)), 0, None, 'Integer') #jumlah yang disimpan pada periode ke i
production = LpVariable.dicts('Prod',list(range(1,13)),0, None, 'Integer') #jumlah yang diproduksi pada periode ke i
binary = LpVariable.dicts('binary', list(range(1,13)), 0, None, 'Binary')# menyatakan apakah akan dilakukan produksi atau tidak pada periode ke i 
time = list(param.index)

#di awal produksi tersedia 200 unit produk
inventory[0] = 200

### 4. Menginisiasi Model
Karena tujuan dari penjadwalan produksi adalah meminimalkan biaya, maka gunakan **LpMinimize**

In [7]:
#mnginisiasi model
model = LpProblem("Production_Scheduling", LpMinimize)

### 5. Menambahkan Fungsi Tujuan dan Constrain
Fungsi tujuan dari permasalahan ini yaitu meminimalkan total biaya meliputi biaya tetap, biaya variable, dan biaya penyimpanan di semua periode

Batasan yang ada pada permasalahan ini yaitu:
1. Jumlah produk yang tersedia pada periode ke i (dari produksi maupun sisa di periode sebelumya) paling tidak harus melebihi demand pada periode ke i. Jika dikurangi demand dan masih sisa akan disimpan untuk periode selanjutnya. 
2. Batasan selanjutnya memastikan apabila binary = 0 maka jumlah produk yang diproduksi akan sama dengan nol sehingga menggunakan persediaan pada periode sebelumnya untuk memenuhi demand.

In [8]:
#fungsi tujuan 
model += lpSum([inventory[t]*param.loc[t,'storage cost'] +
                production[t]*param.loc[t,'var'] +
                binary[t]*param.loc[t,'fixed cost']
                for t in time])

#constrain
for t in time:
    model += production[t] - inventory[t] + inventory[t-1] >= param.loc[t,'demand']
    model += production[t] <= binary[t]*param.loc[t,'Capacity']

### 6. Menyelesaikan Model serta Mengecek Hasil Optimasi

In [9]:
#cek status model -> jika nilainya 1 artinya hasil sudah optimal
model.solve()

1

In [10]:
#cek biaya yang dibutuhkan
model.objective.value()

1373400

Selanjutnya kita akan menggabungkan hasil optimasi jumlah unit yang diproduksi, jumlah unit produk yang disimpan serta status produksi pada masing-masing periode. Kita akan menggunakan bantuan library pandas untuk merapikan data-data tersebut dalam bentuk tabel.

In [11]:
#list hasil optimasi pada setiap periode
unit_producted = [production[i].varValue for i in production]
unit_stored = [inventory[i].varValue for i in range(1,13)]
product_or_not = [binary[i].varValue for i in binary]

In [12]:
answer = pd.DataFrame({'Production':unit_producted,
                       'Inventory':unit_stored,
                       'Product or not':product_or_not}, index=range(1,13))
answer

,Production,Inventory,Product or not
1,3400,600,1
2,4000,600,1
3,4000,2100,1
4,4000,2100,1
5,4000,100,1
6,0,0,0
7,3250,3000,1
8,4000,2000,1
9,4000,0,1
10,1700,900,1


In [13]:
#Untuk hasil yang lebih lengkap

Combine = param.join(answer)
Combine

,period,demand,fixed cost,var,storage cost,Capacity,Production,Inventory,Product or not
t,,,,,,,,,
1,period 1,3000,2000,40,1,4000,3400,600,1
2,period 2,4000,2000,40,1,4000,4000,600,1
3,period 3,2500,2000,40,1,4000,4000,2100,1
4,period 4,4000,2000,40,1,4000,4000,2100,1
5,period 5,6000,2000,40,1,4000,4000,100,1
6,period 6,100,2000,40,1,4000,0,0,0
7,period 7,250,2000,40,1,4000,3250,3000,1
8,period 8,5000,2000,40,1,4000,4000,2000,1
9,period 9,6000,2000,40,1,4000,4000,0,1


Dari tabel di atas kita tahu bahwa pada bulan ke 6 dan 11 tidak dilakukan kegiatan produksi sehingga memanfaatkan persedian pada bulan sebelumnya. Pada akhir periode tidak ada unit produk yang disimpan. Dari hasil optimasi diperoleh biaya sebesar $1.373.400